<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# word -> pdf 파일로 변환(로컬)
# Excel -> pdf 파일로 변환(로컬)
# HWP -> PDF 파일로 변환(로컬)
# 모든 파일은 pdf로 저장
# 모든 html 페이지는 md로 저장
# jpg, pdf(사진) -> ocr로 텍스트 추출 후 저장

"""
앞으로 할일
# 워드 파일 -> pdf 파일로 변환 예시(진행 중)
# 엑셀 파일 -> pdf 변환 해야함(진행 중)
# pdf(사진) -> pdfplumber로도 추출이 안되면 사진이니 ocr 진행하기(완료)
# jpg, png 사진파일 -> 텍스트 추출 후 저장 (완료)
전처리 + 문서 청킹(나누기) 방법 (진행 중)

(미래)
1단계 개선안 (빠르고 간단)
→ collection은 그대로 두고, metadata에 학과/카테고리를 꼼꼼히 저장하고 필터 검색을 잘 활용.
2단계 확장안 (100만 개 이상 규모에서 권장)
→ 학과 또는 카테고리 기준으로 collection을 나누고, 질문 전에 해당 collection만 검색.
사용자 질문 -> 잘 변환 -> 해당되는 collection을 불러와 안에 청크들을 조회하며 유사도 높은걸 검색한다.
"""

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 설치
!pip install chromadb
!pip install sentence-transformers
!pip install pandas
!pip install PyPDF2
!pip install pdfplumber


# ocr PaddleOCR 사용
!pip install paddleocr
!pip install "paddlepaddle==2.6.2" -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install matplotlib

In [ ]:
"""
# transformers 버전이 꼬이면 오류나면 다시 설치
# 1. transformers 삭제
!pip uninstall -y transformers

# 2. 안정적인 버전으로 재설치 (4.40.2 또는 4.41.1 권장)
!pip install transformers==4.41.2
"""

In [11]:
# 구글 드라이브 document_files에 있는 pdf, md 파일들 전부 저장
# 경고 및 오류 처리 방법
# incorrect startxref pointer(1) : Xref Table (객체 위치 정보) 를 찾기 위한 포인터가 잘못되었거나 손상된 경우 나타나는 경고, 되긴하나 일부 페이지나 객체 누락될 수 있음 -> 해결책 pdfplumber
# Advanced encoding /KSCms-UHC-H not implemented yet : DF가 한글 문자셋(KSCms-UHC-H) 을 사용하는데, PyPDF2 라이브러리는 이 고급 인코딩을 완벽하게 지원하지 않는다는 경고 -> 해결책 pdfplumber
# ⚠️ 2020-1 수강신청 안내문(컴퓨터공학전공).pdf: 텍스트 없음 → 스킵 pdf가 사진으로 되어 있는 경우 발생하는 문제 -> 해결책 ocr
import io, os, re, unicodedata, pytz, warnings, logging
import pandas as pd
import numpy as np
import pdfplumber
from datetime import datetime
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
from paddleocr import PaddleOCR
from pdf2image import convert_from_path

warnings.filterwarnings("ignore", category=UserWarning, module='pdfminer')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
ROOT_FOLDER = "/content/drive/MyDrive/document_files"     # 여기만 바꾸기(경로)
URL_MAPPING_SUFFIX = "_url.xlsx"

# PaddleOCR 초기화 (최초 1회만)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

KST = pytz.timezone('Asia/Seoul')
model = SentenceTransformer(EMBEDDING_MODEL_NAME)
client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

# 텍스트 전처리 함수
def clean_text(text):
    # URL이 포함된 문장 제거 (http, https 모두 대응)
    text = re.sub(r'[^.!?\n]*https?://\S+[^\n.?!]*[.!?\n]?', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def split_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

# OCR 함수(사진 파일 .jpg, .jpeg, .png)
def extract_text_from_image(image_path):
    try:
        from PIL import Image
        img = Image.open(image_path).convert("RGB")
        image_np = np.array(img)[:, :, ::-1]  # RGB → BGR
        result = ocr.ocr(image_np, cls=True)
        text = "\n".join([line[1][0] for line in result[0]])
        if text.strip():
            print(f"🖼️ {os.path.basename(image_path)}: 이미지 OCR 추출 성공")
        return text
    except Exception as e:
        print(f"❌ [이미지 OCR 실패] {image_path}: {e}")
        return ""

# OCR 함수(pdf에 텍스트말고 사진이 저장된 경우)
def extract_text_with_paddleocr(pdf_path):
    try:
        images = convert_from_path(pdf_path, dpi=300, poppler_path="/usr/bin")
        full_text = ""
        for i, page in enumerate(images):
            image_np = np.array(page)[:, :, ::-1]  # RGB -> BGR
            result = ocr.ocr(image_np, cls=True)
            for line in result[0]:
                full_text += line[1][0] + "\n"
        return full_text
    except Exception as e:
        print(f"❌ [OCR 실패] {os.path.basename(pdf_path)}: {e}")
        return ""

# ✅ PDF 텍스트 추출 함수 (PyPDF2 -> pdfplumber -> ocr 순으로 모든 pdf 파일 텍스트 추출)
def read_file(filepath):
    def try_pypdf2(path):
        class SkipHandler(logging.Handler):
            def emit(self, record):
                if "incorrect startxref pointer" in record.getMessage():
                    raise ValueError("skip_due_to_incorrect_startxref")

        handler = SkipHandler()
        logger = logging.getLogger("PyPDF2._reader")
        logger.addHandler(handler)
        logger.setLevel(logging.WARNING)

        try:
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")
                reader = PdfReader(path)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
                for warning in w:
                    msg = str(warning.message)
                    if "KSCms-UHC-H" in msg:
                        print(f"⚠️ {os.path.basename(path)}: 한글 글꼴 미지원 (KSCms-UHC-H) → pdfplumber 시도")
                        return ""
                return text
        except ValueError as ve:
            if "skip_due_to_incorrect_startxref" in str(ve):
                print(f"⚠️ {os.path.basename(path)}: 구조 손상 (startxref) → pdfplumber 시도")
                return ""
            else:
                raise
        except Exception as e:
            print(f"❌ [PyPDF2 실패] {path}: {e}")
            return ""
        finally:
            logger.removeHandler(handler)

    def try_pdfplumber(path):
        try:
            import pdfplumber
            with pdfplumber.open(path) as pdf:
                text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                if text.strip():
                    print(f"📄 {os.path.basename(path)}: pdfplumber 추출 성공")
                return text
        except Exception as e:
            print(f"❌ [pdfplumber 실패] {path}: {e}")
            return ""

    if filepath.endswith(".pdf"):
        text = try_pypdf2(filepath)
        if not text.strip():
            text = try_pdfplumber(filepath)
        return text

    elif filepath.endswith(".md"):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()
        except:
            return ""

    elif filepath.lower().endswith(('.jpg', '.jpeg', '.png')):
        return extract_text_from_image(filepath)

    return ""

def get_all_files(folder):
    all_files = []
    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            if f.endswith(('.pdf', '.md', '.jpg', '.jpeg', '.png')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

def extract_category(file_path, base_folder):
    rel_path = os.path.relpath(file_path, start=base_folder)
    parts = rel_path.split(os.sep)
    return parts[0] if len(parts) > 1 else "기타"

def build_metadata(file_path, base_folder, base_name, source_url, ext):
    category = extract_category(file_path, base_folder)
    department_path = os.path.relpath(base_folder, ROOT_FOLDER)

    # 확장자 기반으로 source_type 분기
    if ext == ".pdf":
        source_type = "pdf"
    elif ext == ".md":
        source_type = "markdown"
    elif ext in [".jpg", ".jpeg", ".png"]:
        source_type = "image"
    else:
        source_type = "etc"  # 혹시 모를 확장자에 대한 대비

    return {
        "file_name": base_name,
        "department": department_path,
        "category": category,
        "source_type": source_type,
        "source_url": source_url,
        "date": datetime.now(KST).isoformat()
    }

total_added = 0

# 엑셀 기준으로 저장
for dirpath, _, filenames in os.walk(ROOT_FOLDER):
    for filename in filenames:
        if filename.endswith(URL_MAPPING_SUFFIX):
            folder = dirpath  # 엑셀 위치 폴더 = 기준 폴더
            dept_folder_name = os.path.basename(folder)
            url_file = os.path.join(folder, filename)

            try:
                url_df = pd.read_excel(url_file)
                url_mapping = dict(zip(url_df['파일명'], url_df['URL']))
            except Exception as e:
                print(f"❌ URL 매핑 실패: {url_file} ({e})")
                continue

            file_paths = get_all_files(folder)

            for file_path in file_paths:
                file_name = os.path.basename(file_path)
                base_name = unicodedata.normalize("NFC", os.path.splitext(file_name)[0])
                ext = os.path.splitext(file_name)[1].lower()

                if file_name.endswith(URL_MAPPING_SUFFIX):
                    continue

                # 텍스트 추출 결과가 공백이거나 빈 문자열일 경우 해당 pdf 파일은 사진으로 구성된 경우임
                raw_text = read_file(file_path)
                if not raw_text.strip():
                    print(f"⚠️ {file_name}: 텍스트 없음 → PaddleOCR 시도")
                    raw_text = extract_text_with_paddleocr(file_path)

                # 그래도 텍스트 없으면 최종 스킵
                if not raw_text.strip():
                    print(f"⛔ {file_name}: PyPDF2, pdfplumber, PaddleOCR 방법으로 텍스트 추출 실패 → 스킵")
                    continue

                cleaned = clean_text(raw_text)
                chunks = split_chunks(cleaned)
                embeddings = model.encode(chunks).tolist()
                ids = [f"{dept_folder_name}_{base_name}_chunk_{i}" for i in range(len(chunks))]

                new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

                for chunk, emb, id_ in zip(chunks, embeddings, ids):
                    if id_ not in existing_ids:
                        meta = build_metadata(file_path, folder, base_name, url_mapping.get(base_name, "출처 URL 없음"), ext)
                        new_chunks.append(chunk)
                        new_embeddings.append(emb)
                        new_ids.append(id_)
                        new_metadatas.append(meta)

                if new_chunks:
                    collection.add(
                        documents=new_chunks,
                        embeddings=new_embeddings,
                        metadatas=new_metadatas,
                        ids=new_ids
                    )
                    print(f"✅ {file_name}: {len(new_chunks)}개 저장 완료")
                    total_added += len(new_chunks)
                else:
                    print(f"🟦 {file_name}: 중복 청크 존재 → 스킵")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")

[2025/04/11 08:22:03] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch

In [17]:
import random
from chromadb import PersistentClient

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 전체 문서 불러오기
results = collection.get(limit=None, include=["documents", "metadatas", "embeddings"])
total_docs = len(results['ids'])

print(f"\n📦 전체 저장된 벡터 수: {total_docs}")
print("=" * 70)

# 10개 무작위 추출
sample_indices = random.sample(range(total_docs), min(10, total_docs))

for i in sample_indices:
    doc_id = results["ids"][i]
    metadata = results["metadatas"][i]
    doc_text = results["documents"][i].strip().replace("\n", " ")
    embedding = results["embeddings"][i]

    print(f"🔹 ID: {doc_id}")
    print(f"   📁 파일명: {metadata.get('file_name')}")
    print(f"   🏫 부서: {metadata.get('department')}")
    print(f"   📂 카테고리: {metadata.get('category')}")
    print(f"   📄 타입: {metadata.get('source_type')}")
    print(f"   🌐 URL: {metadata.get('source_url')}")
    print(f"   📅 날짜: {metadata.get('date')[:10]}")
    print(f"   💬 청크 문장: {doc_text[:200]}...")  # 200자까지만 표시
    print(f"   🧠 벡터 길이: {len(embedding)}")
    print(f"   🔢 벡터 앞 5개 값: {embedding[:5]}")
    print("-" * 70)


📦 전체 저장된 벡터 수: 907
🔹 ID: Software Department (소프트웨어학과)_1. 2020학년도 전공관련 직무역량 강화 지원 안내(학생)_chunk_1
   📁 파일명: 1. 2020학년도 전공관련 직무역량 강화 지원 안내(학생)
   🏫 부서: Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)
   📂 카테고리: 커뮤니티(학과공지)
   📄 타입: pdf
   🌐 URL: 출처 URL 없음
   📅 날짜: 2025-04-11
   💬 청크 문장: 율을반영,신청순(증빙완료)으로순위결정 ○ 내용: 지원기준충족학생에게개인부담금총수강료의 50%(최대 200천원)를 장학금으로지원 ○기준 - (유형Ⅰ-수강료지원 ) 개인부담금으로강좌수강후자격증 * 취득 ※붙임1-전공연계직무자격증에한함 - (유형Ⅱ-응시료지원 ) 별도의강좌수강없이자격증 * 취득자 ※응시료(5만원이상)기준은수요조사에제출된자격증 - (유형Ⅲ-임고강좌...
   🧠 벡터 길이: 384
   🔢 벡터 앞 5개 값: [ 0.15552625  0.09100825  0.05141438 -0.0060298  -0.04002262]
----------------------------------------------------------------------
🔹 ID: Software Department (소프트웨어학과)_도서관 전자자료 온라인 이용교육 안내(5월)_chunk_0
   📁 파일명: 도서관 전자자료 온라인 이용교육 안내(5월)
   🏫 부서: Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)
   📂 카테고리: 커뮤니티(특강공지)
   📄 타입

In [19]:
"""
# 파일명을 검색해서 청크 조회
import unicodedata

# ✅ 1. 확인하고자 하는 파일명 (확장자 제외)
target_file = "소프트웨어전공 졸업규정"  # 확장자 제거
normalized_target = unicodedata.normalize("NFC", target_file)   # 꼭 정규화 필요

# ✅ 2. ChromaDB 컬렉션 불러오기 (이미 기존 코드에서 client, collection은 존재한다고 가정)
from chromadb import PersistentClient
client = PersistentClient(path="/content/drive/MyDrive/chroma_index")
collection = client.get_collection(name="knu_collection")

# ✅ 3. 전체 ID 목록 중 검색
all_ids = collection.get(limit=None)['ids']
matched_ids = [id_ for id_ in all_ids if normalized_target in id_]

print(f"\n🔍 일치하는 ID 개수: {len(matched_ids)}")

# ✅ 4. 결과 출력
for id_ in matched_ids:
    result = collection.get(ids=[id_])
    print("\n🆔 ID:", id_)
    print("📝 청크 내용:")
    print(result['documents'][0])  # 👉 전체 출력
    print("📎 메타데이터:")
    print(result['metadatas'][0])
"""


🔍 일치하는 ID 개수: 4

🆔 ID: Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_0
📝 청크 내용:
- 7 -소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라야 한다. ② 3~4학년 재학중 공모전 출품을 1회 하여야 한다(출품인 4인까지 인정), 제3조(졸업논문 계획서 제출) ① 졸업논문 제출자격자는 4학년 1학기(최종학기 직전학기 ) 개시 후 2주내에 졸업논문 계획서[별지서식 (1)]를 논문지도교수의 지도를 받아 전공주임에게 제출 하여 승인을 받아야 한다. ② 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . 제4조(졸업논문 제출) ① 지도교수에게 2개 학기 이상 논문 지도를 받고 학술대회 제 1저자 발 표실적이 1건 이상 있어야하며 , 최종학기 수업 3/
📎 메타데이터:
{'category': '커뮤니티(규정자료실)', 'source_url': '출처 URL 없음', 'source_type': 'pdf', 'department': 'Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)', 'date': '2025-04-11T17:27:07.350870+09:00', 'file_name': '소프트웨어전공 졸업규정'}

🆔 ID: Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_1
📝 청크 내용:
4 이전까지 발표하고 , 종강전까지 인쇄본을 제출하여야한다 ② 단독연

In [9]:
# 완전 내부 초기화 ------ 신중히 사용하기 바람 -------
"""
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 컬렉션 내의 모든 문서 ID를 가져와 삭제
existing_ids = collection.get(limit=None)['ids']
if existing_ids:
    collection.delete(ids=existing_ids)
    print(f"🗑️ 컬렉션 내 모든 데이터 삭제 완료: {len(existing_ids)}개 삭제됨")
else:
    print("✅ 삭제할 데이터가 없습니다. 컬렉션이 이미 비어있습니다.")
"""

🗑️ 컬렉션 내 모든 데이터 삭제 완료: 907개 삭제됨


In [ ]:
""" PDF 텍스트 추출 성능 비교 => PyPDF2가 더 잘 나옴
!pip install PyPDF2
!pip install pdfplumber

import PyPDF2
import pdfplumber

with open("/content/drive/MyDrive/document_files/Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)/커뮤니티(규정자료실)/미래설계 교과목 이수기준(2021.04.19.).pdf", "rb") as f:
    reader = PyPDF2.PdfReader(f)
    print("[PyPDF2]")
    print(reader.pages[0].extract_text())

with pdfplumber.open("/content/drive/MyDrive/document_files/Department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)/커뮤니티(규정자료실)/미래설계 교과목 이수기준(2021.04.19.).pdf") as pdf:
    print("\n[pdfplumber]")
    print(pdf.pages[0].extract_text())
"""

In [ ]:
""" 예제 코드에서 사용할 OCR 설치
# Tesseract OCR => 추출이 이상하게 됨
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!pip install pytesseract pdf2image Pillow
"""

""" 이거 사용함
# PaddleOCR 설치 (최초 1회) => 한글 지원 매우 좋고, 정확도도 높음, 딥러닝 기반 고정밀 OCR
!pip install paddleocr
!pip install "paddlepaddle==2.6.2" -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install matplotlib
"""

In [20]:
from PIL import Image
import pytesseract

# 이미지 경로
IMG_PATH = "/content/drive/MyDrive/sample/공결규정_사본.jpg"

# 이미지 로드 (전처리 없음)
img = Image.open(IMG_PATH)

# OCR 수행
text = pytesseract.image_to_string(img, lang="kor+eng", config="--psm 6 --oem 3")

# 결과 출력
print("🖼️ 이미지 OCR 추출 결과:\n")
print(text)

🖼️ 이미지 OCR 추출 결과:

<4 294 Ba FASTA a GALE TA Us BAD
Ac2z(SA YP YA) OS 4 Sel HSH AS, BY SYASS Ast Fe sdagaqal As
AAD Haye AA soll Aeeto She AF SAE Usa edt Ale Ed,
(S42) 1 BAUS [et dels DW UYASA, AIA S WAIF] Us a4 MS It
[34] 2 Az4e Ws a4
a
a
=
se
°
BBA 2S 2s SHA BHM YS SAA+ol elstz] Pol, Sersto, ase + act.
(Sa) 3 S30 44 ASue Ate Wet AA: Bt Tt
[BA] 4 At DRI olFsh Beate] @Useol Bist Aeagd|s, Weds, ag sy: a
S 7a
[34] 5 ASS2+4 WHSA SOE Vt AASGAS Hel Ay 2E9 172|Bt HS): WBSA |Z,
¥d 12, aisle ae ose
(S24) 6 JA5 AHel He} Aer Bast Agel Au aH: My Aaaiz
[22] 11 Yoel S40] Paatchan Waste ae
42] O84 EE VISE Uda Hee +9 + Ud AAT AF Be SUASS ast We
SUNBAA Ase AADe Hate yte Aa Hse Aste See Wole BSE sls s
SUS avel 1 olsa|e wae delata S408 ogee
<4 294 AQ Ba 7] A1y>
- BEVIS Basta Ute AHye GE SA BYAFA): GH SECSsaa BH), Ga
A MGS)
- TH2Y EE Boe Ut Bee Bee sg 2 Sse azo iolealet YE APT
ZA OR UMtH UES 153 TE AS AAD YAAte Ape ys seel ast
- Ba SY wdol oeisiel BF Hl WUE YS Seto BE SYS Ssatofok HSyol Bast
AF we)



In [18]:
from pdf2image import convert_from_path
import pytesseract

# 설정
POPPLER_PATH = "/usr/bin"
PDF_PATH = "/content/drive/MyDrive/sample/2. 홍보자료_ICT 융합인재 양성과정.pdf"
LANG = "kor+eng"

# PDF → 이미지 변환
images = convert_from_path(PDF_PATH, dpi=300, poppler_path=POPPLER_PATH)

# OCR 실행, 이미지 전처리 없어 기본 모델 성능만으로 텍스트 추출
for i, page in enumerate(images):
    text = pytesseract.image_to_string(page, lang=LANG)
    print(f"\n--- Page {i+1} ---")
    print(text)


--- Page 1 ---
ae BAS oes

Tage Stapemlpoe ps

 

Ske)
foo ee NTU Ue eo ba a
A!Z7|ZE | 2020. 06. 01.(B) ~ 2020. 06. 19.(e)
arse oe laa SI
eS ae Bt
pr a od bk. Ae) Rae AG m9 MIO Veo

i174 a0 oN (3 Rage”. a (=) Bok) SD)
PES os ELT stages eae
a a Lot 0 ar eS eS Se ca
= F | SEHStd BABA (041-850-0432~3)

  


--- Page 2 ---
IcT Setelay SSarg
SSS BHl0|0| Peizey

t

oleh
AR

22 ¢
Busty

CV;

SAAS MAAA! AI
Open

cat 1 Y,

x
=<
ou
me
2
ui
oO
|
(=)

| MH|AE Cf

S| ABA!

2

~
3

 

I

 

un S

OpencVv
Sa Bei

Ee
So

ny
1
aa
ol
io,
ol
nh
K
a
<I
oO
nl
iz
Kr
Te
I
6
ny

AG" Of CHH|St AIAeo

BAF

R&D

 


--- Page 3 ---
7

arg
ars

OFA
oo
5

SHICT Satelay
OpenCve BsstajAn SSS Sei/0/0] +

t

OF
oll
or
K
RJ
yor
Kt
KR
or
K
0
{0

 

po ae be

ky

ke

ES

Azz

i

°o

OpenCV 28 Video Player=

0

64

Tee!

Opencvatesilol 4

CHa

e

A
Tene BEBO Ay 7|

SsAe| 7 | Bt AAA CIA!

SS 7A Zea

ez

 

 


--- Page 4 ---
7

Saar
are

tOlxy
H]0|0} +7

ox
Sa
Ze

FCT

gs
The
HAM SSS

oJ

ce
a
et

Bus

K
Zu
yor
ae
zg
ao

In [21]:
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np

# OCR 초기화 (한글 지원)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

# 이미지 경로
IMG_PATH = "/content/drive/MyDrive/sample/공결규정_사본.jpg"

# 이미지 로드 (전처리 없음)
img = Image.open(IMG_PATH)
image_np = np.array(img.convert("RGB"))[:, :, ::-1]  # PIL → NumPy(BGR)

# OCR 수행
result = ocr.ocr(image_np, cls=True)

# 결과 출력
print("🖼️ 이미지 OCR 추출 결과:\n")
for line in result[0]:
    text = line[1][0]
    print(text)

[2025/04/11 06:30:16] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch

In [19]:
from paddleocr import PaddleOCR
from pdf2image import convert_from_path
import numpy as np

# 경로 설정
POPPLER_PATH = "/usr/bin"
PDF_PATH = "/content/drive/MyDrive/sample/2. 홍보자료_ICT 융합인재 양성과정.pdf"

# OCR 초기화 (한글 지원)
ocr = PaddleOCR(use_angle_cls=True, lang='korean')

# PDF → 이미지 변환
images = convert_from_path(PDF_PATH, dpi=300, poppler_path=POPPLER_PATH)

# OCR 실행, 이미지 전처리 없어 기본 모델 성능만으로 텍스트 추출
full_text = ""
for i, page in enumerate(images):
    image_np = np.array(page)[:, :, ::-1]  # PIL → numpy (BGR)

    result = ocr.ocr(image_np, cls=True)

    full_text += f"\n--- Page {i+1} ---\n"
    for line in result[0]:
        full_text += line[1][0] + "\n"

# 결과 출력
print("📄 전체 OCR 텍스트 결과:\n")
print(full_text)

[2025/04/11 06:26:53] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch